In [ ]:
# --------starts here-------
# --------read-----------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sys,os
from seglearn.transform import SegmentX
from seglearn.feature_functions import maximum,minimum
from sklearn.preprocessing import normalize,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import svm

skip_list = [2,4,10,12,20]

y=[]

for k in range(10):
    for i in range(22):
        if i in skip_list:
            continue
        for j in range(10):
            y.append(i)

# for i in range(22):
#     for j in range(30):
#         y.append(i)

print(y)

tsfresh_feature = pd.read_csv("data/features_filtered_user.csv")

tsfresh_feature = tsfresh_feature.drop(["id"],axis=1)
# tsfresh_feature = tsfresh_feature.drop([i for i in range(7*220,(7+1)*220)])

for i in range(10):
    for skipping in skip_list:
        try:
            tsfresh_feature = tsfresh_feature.drop([skipping*10+j+i*220 for j in range(10)])
        except:
            print(i)
# for idx in range(10):

X = tsfresh_feature[:]
y = y
print(X.shape)
maxyhat_train=0.0
maxyhat_test=0.0
temp = 0
feature_list=[]
result_list = np.zeros(100)
train_results = []
test_results = []
for i in range(10,20):
    for j in range(10,20):
        x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=i, train_size=0.75, test_size=0.25)

        clf = RandomForestClassifier(n_estimators=100, max_depth=30, random_state=j)
        clf.fit(x_train, y_train)
        temp = clf.score(x_train, y_train)
        train_results.append(temp)
        if (temp > maxyhat_train).any():
            maxyhat_train = temp
        temp = clf.score(x_test, y_test)
        test_results.append(temp)
        feature_importances = pd.DataFrame(clf.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
        result_list[(i-10)*10+(j-10)] = temp
        if (maxyhat_test < temp).any():
            maxyhat_test = temp
            for feature in feature_importances.index.tolist()[:50]:
                if not feature in feature_list:
                    feature_list.append(feature)

                    
print(maxyhat_train)
print(maxyhat_test)

from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(np.arange(100), train_results,'b', label='Train AUC')
line2, = plt.plot(np.arange(100), test_results,'r', label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('n_estimators')
plt.show()

In [ ]:
#------ using ranked features ----------#
train_results = []
test_results = []
for i in range(10,20):
    for j in range(10,20):
        x_train, x_test, y_train, y_test = train_test_split(X[feature_list], y, random_state=i, train_size=0.8, test_size=0.2)
        clf = RandomForestClassifier(n_estimators=100, max_depth=30, random_state=j)
#         clf = svm.SVC(C=0.4,kernel='linear',gamma=5)
        clf.fit(x_train, y_train)
        temp = clf.score(x_train, y_train)
        train_results.append(temp)
        if (temp > maxyhat_train).any():
            maxyhat_train = temp
        temp = clf.score(x_test, y_test)
        test_results.append(temp)
        if (maxyhat_test < temp).any():
            maxyhat_test = temp

from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(np.arange(100), train_results,'b', label='Train AUC')
line2, = plt.plot(np.arange(100), test_results,'r', label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('n_estimators')
plt.show()
print(maxyhat_train)
print(maxyhat_test)

In [ ]:
#------ adding features from Shutong ------#
# fftfeatures1 = np.loadtxt(fname = "data/22gesture.txt",delimiter=',')
# fftfeatures = np.loadtxt(fname = "data/qstSpectro.txt",delimiter=',')
# fftfeatures = np.concatenate((fftfeatures1,fftfeatures2),axis=0)
fftfeatures = np.loadtxt(fname = "data/userStudy.txt",delimiter=',')
print(fftfeatures.shape)
_len = len(fftfeatures[:][0])
fft_df = pd.DataFrame(data=fftfeatures[:],    # values
              index=[i for i in range(2200)],    # 1st column as index
              columns=["f"+str(i) for i in range(_len)])

fft_df = fft_df.drop([i for i in range(7*220,(7+1)*220)])

for i in range(10):
    for skipping in skip_list:
        try:
            fft_df = fft_df.drop([skipping*10+j+i*220 for j in range(10)])
        except:
            print(i)

features_combine = fft_df.join(X[feature_list])
features_combine.shape

In [ ]:
#------------ Random Forest -------------#
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import serial
import sys,os
from seglearn.transform import SegmentX
from seglearn.feature_functions import maximum,minimum
from sklearn.preprocessing import normalize,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import svm

X = features_combine
print(X.shape)
maxyhat_train=0.0
maxyhat_test=0.0
temp = 0


result_list = np.zeros(100)
train_results = []
test_results = []
for i in range(10,20):
    for j in range(10,20):
        x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=18, train_size=0.8, test_size=0.2)

        clf = RandomForestClassifier(n_estimators=100, max_depth=50, random_state=j)
#         clf = svm.SVC(C=0.4,kernel='linear',gamma=5)
        clf.fit(x_train, y_train)
        temp = clf.score(x_train, y_train)
        train_results.append(temp)

        if (temp > maxyhat_train).any():
            maxyhat_train = temp
        #show_accuracy(y_hat, y_train, '训练集')
        temp = clf.score(x_test, y_test)
        test_results.append(temp)

#         print(temp)

        feature_importances = pd.DataFrame(clf.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
        result_list[(i-10)*10+(j-10)] = temp
        if (maxyhat_test < temp).any():
            maxyhat_test = temp
            for feature in feature_importances.index.tolist()[:50]:
                if not feature in feature_list:
                    feature_list.append(feature)
            print(len(feature_list))
            
        #show_accuracy(y_hat, y_test, '测试集')

from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(np.arange(100), train_results,'b', label='Train AUC')
line2, = plt.plot(np.arange(100), test_results,'r', label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('n_estimators')
plt.show()

print(maxyhat_train)
print(maxyhat_test)

train_results
test_results


In [ ]:
#------ using ranked features ----------#
bestmodel = 0
maxyhat_test=0
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

#     print(cm)

    fig, ax = plt.subplots(figsize=(15,15))
    
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

v2_names = ["button_on", "button_off", "calling", "check", "circle", "double_tap", "fast_swipe", "finger_press", "finger_rub", "finger_slide",
        "hand_rotation", "hold", "horizontal_swipe", "palm_tilt", "pull", "push", "ranctangle", "round", "single_tap", "slow_swipe",
        "snapping_finger", "triangle"]

v2_names = np.array(v2_names)
y_pred = None
for i in range(10,20):
    for j in range(10,20):
        x_train, x_test, y_train, y_test = train_test_split(X[feature_list], y, random_state=i, train_size=0.9, test_size=0.1)
#         x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=i, train_size=0.8, test_size=0.2)

        clf = RandomForestClassifier(n_estimators=100, max_depth=50, random_state=j)
#         clf = svm.SVC(C=0.4,kernel='linear',gamma=5)
        model = clf.fit(x_train, y_train)
        temp_train = clf.score(x_train, y_train)
        
        temp_test = clf.score(x_test, y_test)
        if (maxyhat_test < temp_test).any():
            y_pred = clf.predict(x_test)
            maxyhat_train = temp_train
            maxyhat_test = temp_test
            plot_confusion_matrix(y_test, y_pred, classes=v2_names, normalize=True,
                      title='Normalized confusion matrix')
            plt.show()
            print(maxyhat_test)
print(maxyhat_train)
print(maxyhat_test)